In [1]:
## Step 2: Backtest framework

# You will generate a signal:

#1 = long
#0 = cash
#The backtest uses position = signal.shift(1) to avoid look-ahead bias.

# Define the function which will develop


## Step 3: Implement one strategy

# Implement **ONE** of the following by producing a `signal` series (0/1). Comment out the options you are not using.

# **Guidance:** Choose which strategy option (A1, A2, or A3) you will implement and use only that signal. For whichever strategy you pick, decide on appropriate parameter values:
#- If using **ROC Threshold**: select the lookback window `roc_n` and a percentage threshold. (e.g., `roc_n=10` and `roc_threshold=0.03` means go long if price has risen more than 3% over the last 10 days.)
#- If using **MA Crossover**: choose short (`fast_n`) and long (`slow_n`) moving average lengths. (A typical choice might be 20-day vs 50-day MAs, but you can experiment.)
#- If using **MACD**: you can use the standard periods (12, 26, 9) for fast EMA, slow EMA, and signal line, or other values if justified.

#> **Checkpoint 2:** After developing one trading strategy, considering committing your notebook (e.g., "Completed ROC Threshold").

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf

# Load Data
# Download price data for AAPL
ticker = "AAPL"
start_date = "2021-01-01"
end_date = "2026-01-01"

price_data = yf.download(ticker, start_date, end_date)
price_data = price_data[['Close']]

print(price_data.head().round(2))

# turn it into an entire function - pass in different 

[*********************100%***********************]  1 of 1 completed

Price        Close
Ticker        AAPL
Date              
2021-01-04  125.97
2021-01-05  127.53
2021-01-06  123.24
2021-01-07  127.44
2021-01-08  128.54


In [3]:
## STEP 2: The Backtest Framework

# Create a column for signals and fill with 0 (In cash default)
# Creating the container for our strategy, for when we calculate ROC in Step3 
price_data['Signal'] = 0

# Create column for position
# This creates a NEW column named position, taking the signal column and moving everything down 1 row
price_data['Position'] = price_data['Signal'].shift(1)

#Prevents 'look-ahead bias'
# The quotation marks means Python treats it as a label (string of text) Without them, python thinks it is a variable. 

In [4]:
def roc_threshold_strategy(data, roc_n = 10, threshold = 0.03):
    data = data.copy()
    momentum = data['Close'].pct_change(periods = roc_n)
    signal = np.where(momentum > threshold, 1, 0)
    data["ROC_signal"]=signal
    return data

#backtest framework redone to avoid calculations twice
signal = roc_threshold_strategy(price_data, roc_n = 10, threshold = 0.03)

signal['Position'] = signal['ROC_signal'].shift(1)
print(signal)

Price            Close Signal Position ROC_signal
Ticker            AAPL                           
Date                                             
2021-01-04  125.974449      0      NaN          0
2021-01-05  127.531952      0      0.0          0
2021-01-06  123.239059      0      0.0          0
2021-01-07  127.444389      0      0.0          0
2021-01-08  128.544388      0      0.0          0
...                ...    ...      ...        ...
2025-12-24  273.809998      0      0.0          0
2025-12-26  273.399994      0      0.0          0
2025-12-29  273.760010      0      0.0          0
2025-12-30  273.079987      0      0.0          0
2025-12-31  271.859985      0      0.0          0

[1255 rows x 4 columns]
